# Decision Tree Model

In [1]:
# import necessary packages

import pandas as pd
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
# TODO: modell mit eurem ersetzen
from sklearn.tree import DecisionTreeRegressor

In [2]:
# read data

train_df = pd.read_csv('data/preprocessed_data/train.csv')
test_df = pd.read_csv('data/preprocessed_data/test.csv')

In [3]:
train_df.sort_values('start_date')

,start_date,latitude,longitude,month,weekday,pm,mean_temperature,total_precipitation,distance_to_center,holiday,count
0,2017-04-15 00:00:00,45.533703,-73.515283,4,5,0,11.3,3.1,5.346598,False,1
349,2017-04-15 00:00:00,45.521116,-73.594784,4,5,0,11.3,3.1,2.813397,False,6
347,2017-04-15 00:00:00,45.518290,-73.592350,4,5,0,11.3,3.1,2.460816,False,10
650,2017-04-15 00:00:00,45.456355,-73.597557,4,5,0,11.3,3.1,5.702728,False,3
345,2017-04-15 00:00:00,45.517000,-73.589000,4,5,0,11.3,3.1,2.171792,False,14
...,...,...,...,...,...,...,...,...,...,...,...
419347,2018-10-31 12:00:00,45.546978,-73.575515,10,2,1,2.4,8.2,4.855448,False,5
418959,2018-10-31 12:00:00,45.534563,-73.554786,10,2,1,2.4,8.2,3.615471,False,7
419349,2018-10-31 12:00:00,45.545200,-73.576451,10,2,1,2.4,8.2,4.667799,False,10
419341,2018-10-31 12:00:00,45.550316,-73.573358,10,2,1,2.4,8.2,5.210238,False,11


In [4]:
test_df.sort_values('start_date')

,start_date,latitude,longitude,month,weekday,pm,mean_temperature,total_precipitation,distance_to_center,holiday,count
0,2019-04-14 00:00:00,45.549598,-73.541874,4,6,0,5.2,5.4,5.530209,False,5
534,2019-04-14 00:00:00,45.550306,-73.573353,4,6,0,5.2,5.4,5.209004,False,5
536,2019-04-14 00:00:00,45.457509,-73.639485,4,6,0,5.2,5.4,7.517624,False,5
538,2019-04-14 00:00:00,45.557192,-73.569847,4,6,0,5.2,5.4,5.962663,False,8
540,2019-04-14 00:00:00,45.546978,-73.575515,4,6,0,5.2,5.4,4.855448,False,4
...,...,...,...,...,...,...,...,...,...,...,...
232779,2019-10-31 12:00:00,45.524375,-73.591810,10,3,1,12.1,59.7,2.928582,False,13
232781,2019-10-31 12:00:00,45.527698,-73.586253,10,3,1,12.1,59.7,3.009627,False,1
232783,2019-10-31 12:00:00,45.518309,-73.603286,10,3,1,12.1,59.7,3.148699,False,2
232744,2019-10-31 12:00:00,45.544599,-73.588223,10,3,1,12.1,59.7,4.807186,False,7


In [5]:
#TODO: this should be done in preprocessing

# drop start date for regression (is represented as month, day, pm) 
train_df = train_df.drop(columns='start_date')
test_df = test_df.drop(columns='start_date')

# drop null values
train_df = train_df.dropna()
test_df = test_df.dropna()

In [6]:
# split data

X_train = train_df.drop(columns='count')
y_train = train_df['count']

X_val, X_test, y_val, y_test = train_test_split(
    test_df.drop(columns='count'),
    test_df['count'],
    test_size=0.5,
    shuffle=True,
    random_state=7
)

In [15]:
# hyperparameter tuning

try:
    # TODO: file-name ersetzen (in diesem File werden die Ergebnisse des Hyperparameter Tuning gespeichert)
    hyperparameters_df = pd.read_csv('data/hyperparameter_tuning/decision_treeTEST.csv')
    
except FileNotFoundError:
    
    # df containing hyperparameters and evaluation metrics of each run
    hyperparameters_df = pd.DataFrame()
    
    # this function is used by optuna to tune the hyperparameters
    def objective(trial):
        # TODO: die Hyperparameter mit denen eures Modells ersetzen
        # - integers: trial.suggest_int(name, low, high)
        # - floats: trial.suggest_int(name, low, high)
        # - kategorisch: trial.suggest_categorical(name, choices)
        # (https://optuna.readthedocs.io/en/v2.0.0/reference/generated/optuna.trial.Trial.html)
        
        # define hyperparameters
        # criterion=absolute_error takes long to calculate
        #criterion = trial.suggest_categorical('criterion', ['squared_error','friedman_mse','absolute_error','poisson'])
        criterion = trial.suggest_categorical('criterion', ['squared_error','friedman_mse','poisson'])
        splitter = trial.suggest_categorical('splitter', ['best','random'])
        
        # TODO: mit eurem Modell ersetzen
        # setup and train model
        dt_reg = DecisionTreeRegressor(
            criterion=criterion,
            splitter=splitter
        )
        dt_reg.fit(X_train, y_train)
        
        # make predictions
        y_val_pred = dt_reg.predict(X_val)
        
        # evaluate predictions
        r_squared = r2_score(y_val, y_val_pred)
        rmse = mean_squared_error(y_val, y_val_pred) ** 0.5
        
        # TODO: mit euren Hyperparametern ersetzen
        # insert results in dataframe
        global hyperparameters_df 
        hyperparameters_df = hyperparameters_df.append(
            {'criterion': criterion,
             'splitter': splitter,
             'r_squared': r_squared,
             'rmse': rmse},
            ignore_index=True
        )
        
        # return rmse -> optuna will optimize rmse
        return rmse
        
        
    study = optuna.create_study()
    # start optimization
    study.optimize(objective, n_trials=20)
    
    # TODO: evtl. müsst ihr auch noch mal die Datentypen anpassen
    # convert to correct data types
    #hyperparameters_df[['n_estimators', 'max_depth']] = hyperparameters_df[['n_estimators', 'max_depth']].astype('int')
    
    # sort hyperparameter tuning results and save file
    hyperparameters_df = hyperparameters_df.sort_values('rmse', ascending=True)
    hyperparameters_df = hyperparameters_df.reset_index(drop=True)
    hyperparameters_df.to_csv('data/hyperparameter_tuning/decision_tree.csv', index=False)

[I 2022-11-16 22:51:27,404] A new study created in memory with name: no-name-d031428f-5007-467b-90b5-1573f59bcd36
C:\Users\danie\AppData\Local\Temp\ipykernel_10296\1372688550.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hyperparameters_df = hyperparameters_df.append(
[I 2022-11-16 22:51:29,665] Trial 0 finished with value: 17.994436503085687 and parameters: {'criterion': 'squared_error', 'splitter': 'best'}. Best is trial 0 with value: 17.994436503085687.
C:\Users\danie\AppData\Local\Temp\ipykernel_10296\1372688550.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hyperparameters_df = hyperparameters_df.append(
[I 2022-11-16 22:51:30,873] Trial 1 finished with value: 19.88357018886101 and parameters: {'criterion': 'squared_error', 'splitter': 'random'}. Best is trial 0 with value: 17.994436503085687.

C:\Users\danie\AppData\Local\Temp\ipykernel_10296\1372688550.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hyperparameters_df = hyperparameters_df.append(
[I 2022-11-16 22:52:02,736] Trial 18 finished with value: 18.288130028194526 and parameters: {'criterion': 'squared_error', 'splitter': 'random'}. Best is trial 17 with value: 17.0916102834263.
C:\Users\danie\AppData\Local\Temp\ipykernel_10296\1372688550.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hyperparameters_df = hyperparameters_df.append(
[I 2022-11-16 22:52:03,858] Trial 19 finished with value: 18.020859299836893 and parameters: {'criterion': 'squared_error', 'splitter': 'random'}. Best is trial 17 with value: 17.0916102834263.


In [16]:
hyperparameters_df.head()

,criterion,splitter,r_squared,rmse
0,squared_error,random,0.590431,17.091610
1,squared_error,random,0.561220,17.690602
2,friedman_mse,best,0.560933,17.696390
3,friedman_mse,random,0.552505,17.865439
4,squared_error,random,0.550935,17.896746


Note: If splitter=random is optimal hyperparameter, final model evaluation can turn out worse than what was achieved in optimazation (due to randomness).

In [18]:
# final model evaluation

# TODO: mit eurem Modell und Hyperparametern ersetzen
# build and train model using the most successful hyperparameters
dt_reg = DecisionTreeRegressor(
    criterion=hyperparameters_df.loc[0, 'criterion'],
    splitter=hyperparameters_df.loc[0, 'splitter']
)
dt_reg.fit(X_train, y_train)

# make predictions
y_test_pred = dt_reg.predict(X_test)

# evaluate predictions
r_squared = r2_score(y_test, y_test_pred)
rmse = mean_squared_error(y_test, y_test_pred) ** 0.5

print(f'R^2:\t{r_squared}')
print(f'RMSE:\t{rmse}')

R^2:	0.5401148288593869
RMSE:	17.91689743833428
